In [16]:
from transformers import BertForQuestionAnswering, BertTokenizer

In [17]:
def chatbot(text, question):
    model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForQuestionAnswering.from_pretrained(model_name)

    input_ids = tokenizer.encode(question, text)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token itself.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0] * num_seg_a + [1] * num_seg_b

    # Run the model.
    outputs = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    start_score, end_score = outputs.start_logits, outputs.end_logits

    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_score)
    answer_end = torch.argmax(end_score)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Join the tokens to create the answer string.
    answer = ' '.join(tokens[answer_start:answer_end + 1])

    # Remove special tokens and extra spaces.
    answer = answer.replace('[CLS]', '').replace('[SEP]', '').replace(' ##', '').strip()

    return answer

In [22]:
# Example usage:
text = """Your company offers various products such as Product A, Product B, and Product C.
Product A has a battery life of 10 hours and is waterproof.
Product B has a battery life of 8 hours and is not waterproof.
Product C has a battery life of 12 hours and is waterproof."""

question = "Which product runs more than 7 hours"
answer = chatbot(text, question)
print(answer)

product b


In [23]:
len(input())

2201